In [1]:
import numpy as np # numpy used exclusively for np.array's which are more convenient for the purposes of this assignment

# Exercise 1 - Gradient Descent

# Define functions
def f(x: np.array) -> float:
    return x[0]**4 + 4 * x[0] * x[1] + 2 * x[1] + (1/2) * x[1]**2

def grad_f(x: np.array) -> np.array:
    der_1 = 4 * x[0]**3 + 4 * x[1]
    der_2 = 4 * x[0] + x[1] + 2
    return np.array([der_1, der_2])

def eta_const(t: float, c: float = 0.01) -> float:
    return c

def eta_sqrt(t: float, c: float = 0.1) -> float:
    return c / (t + 1)**0.5

def eta_multistep(t: float, milestones: list, c: float = 0.1, eta_init: float = 0.1) -> float:
    if t >= milestones[-1]:
        steps = milestones + [t + 1] # ensure that the loop is always terminated
    else:
        steps = milestones
    eta = eta_init
    i = 0
    while t >= steps[i]:
        eta *= c
        i += 1
    return eta

def gradient_descent(f, grad_f, eta, x_0: np.array, max_iter: int = 100):
    x_t = x_0
    for t in range(max_iter):
        x_tplus1 = x_t - eta(t) * grad_f(x_t)
        x_t = x_tplus1
    return x_t, f(x_t)


In [2]:
x_0 = np.array([1, 1])

# A
result = gradient_descent(f, grad_f, eta_const, x_0)
print(f"after 100 iterations we find x = {result[0]} with function value f(x_100) = {result[1]}")

# B
result = gradient_descent(f, grad_f, eta_sqrt, x_0)
print(f"after 100 iterations we find x = {result[0]} with function value f(x_100) = {result[1]}")

# C
milestones_test = [10,60,90]
result = gradient_descent(f, grad_f, lambda t: eta_multistep(t, milestones = milestones_test, c = 0.5, eta_init = 0.1), x_0)
print(f"after 100 iterations we find x = {result[0]} with function value f(x_100) = {result[1]}")

after 100 iterations we find x = [ 1.46732195 -3.34391166] with function value f(x_100) = -16.087776514949777
after 100 iterations we find x = [ 1.81997381 -6.10987937] with function value f(x_100) = -27.062365302355108
after 100 iterations we find x = [  2.15487448 -10.0170811 ] with function value f(x_100) = -34.64347114072298


In [3]:
# Exercise 2 - Coordinate descent
def f(x: np.array) -> float:
    return (1/2) * x[0]**4 + x[0] * x[1] + x[1]**2 + x[1] * x[2] + x[2]**2

# Note: x[i] = x_i+1, so x[0] = x1 etc
def argmin_x1(x: np.array) -> float:
    return (((1/2) * x[1])**(1/3)).real # Derivative wrt x1, equated to zero
        
def argmin_x2(x: np.array) -> float:
    return (1/2) * (x[0] - x[2]) 

def argmin_x3(x: np.array) -> float:
    return -(1/2) * x[1]

# Fractional powers and np.array elements do not go well together, so a workaround changing dtypes is used
def coordinate_descent(f, argmin, x_0: np.array, max_iter: int = 100) -> np.array:
    x_t = x_0
    for i in range(1, max_iter + 1):
        for j in range(len(x_0)):
            x_t[j] = argmin[j](x_t)
    x_t = x_t.astype('float64')
    return x_t, f(x_t)

In [212]:
argmin = [argmin_x1, argmin_x2, argmin_x3]
x_0 = np.array([5, 10, 5], dtype = complex)

result = coordinate_descent(f, argmin, x_0, max_iter = 100)
print(f"After 100 iterations we find x1 = {result[0][0]}. The other coordinates are x2 = {result[0][1]} and x3 = {result[0][2]}, with corresponding function value f(x_100) = {result[1]}")

After 100 iterations we find x1 = 0.5773502691896257. The other coordinates are x2 = 0.38490017945975047 and x3 = -0.19245008972987523, with corresponding function value f(x_100) = 0.3888888888888888


C:\Users\jespe\AppData\Local\Temp\ipykernel_39028\1276088805.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  x_t = x_t.astype('float64')


In [1]:
# Exercise 3
x1 = [3, -2, 2, -1.5, 1.5, -2.5, -2, 2]
x2 = [1, 2, 2, 2, 3.5, 2.5, 0, 2.5]
y = [0, 1, 0, 1, 0, 1, 1, 0]
predictions = [0 for i in range(len(x1))]

def predict(x1, x2):
    value = -0.078*x1 + -0.227*x2 + 0.165
    if value > 0:
        return 1
    else:
        return 0

for i in range(len(x1)):
    predictions[i] = predict(x1[i], x2[i])

n = 0
correct = 0
incorrect = 0
for i in range(len(x1)):
    if y[i]==0:
        n += 1
        if predictions[i]!=y[i]:
            incorrect += 1
        else:
            correct += 1

print(str(correct/n*100)+"%")

100.0%


Since the answer is equal to 100%, we have that all samples of class 0 in the validation data is correctly classified

In [4]:
# exercise 4
import pandas as pd
import numpy as np

df = pd.read_csv('heart_train_data.csv')

outputDic = {}

xd = df.drop('target', axis=1)

for yValue in df['target'].unique():
    Ic = df['target']==yValue
    tempDic = {}
    for item in xd:
        dic = {}
        for el in xd[item].unique():
            dic[el] = sum(xd[item][Ic]==el)/len(xd[item][Ic])
        tempDic[item] = dic
    outputDic[yValue] = tempDic
pyc = {0:1 - df['target'].mean(), 1:df['target'].mean()}

def predict(input, outputDic, pyc):
    dic = {}
    for item in outputDic:
        prod = pyc[item]
        for el in input:
            prod *= outputDic[item][el][input[el]]
        dic[item] = prod
    max = 0
    for item in dic:
        if max < dic[item]:
            max = dic[item]
            index = item
    return index

df1 = pd.read_csv('heart_validate_data.csv')
matrix = df1.values.tolist()
result = []
for item in matrix:
    pred = predict({"cp": item[0], "exang": item[1], "thal": item[2]}, outputDic, pyc)
    if pred == item[3]:
        result.append(1)
    else:
        result.append(0)

print('Question A')
print(outputDic[1]["thal"][2])
print("Question B")
print(sum(result)/len(result))
print("Question C")
print(df['target'].mean())
print("Question D")
print(predict({"cp":1,"exang":1,"thal":2}, outputDic, pyc))

Question A
0.8290598290598291
Question B
0.7472527472527473
Question C
0.5571428571428572
Question D
1


Since the answer to the last question is 1, we expect the patient to not have the disease, so the answer is D